<a href="https://colab.research.google.com/github/hwasol/esg/blob/main/%EC%A7%80%EC%A0%95%ED%95%99_%EC%A0%81%EA%B0%80%EC%B9%98_%EC%A7%80%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

# 1. 파일 업로드 (Colab에서 파일 직접 업로드)
from google.colab import files
uploaded = files.upload()

# 2. 파일 읽기 및 컬럼명 정리
df_border = pd.read_csv('아프리카_국경공유국수_20112022.csv')[['국가', '2022']].rename(columns={'2022':'국경공유국수'})
df_un = pd.read_csv('아프리카_25개국_UN평화유지군_파견인원_20112022.csv')[['국가', '2022']].rename(columns={'2022':'UN평화유지군'})
df_org = pd.read_csv('아프리카_지역기구활동점수_20112022.csv')[['국가', '2022']].rename(columns={'2022':'지역기구활동점수'})
df_trade = pd.read_csv('아프리카_25개국_국제무역규모_20112022.csv')[['국가', '2022']].rename(columns={'2022':'국제무역규모'})
df_terror = pd.read_csv('아프리카_테러발생건수_20112022.csv')[['국가', '2022']].rename(columns={'2022':'테러발생건수'})

# 3. 테러발생건수는 음수로 변환 (높을수록 나쁨)
df_terror['테러발생건수'] = -1 * df_terror['테러발생건수']

# 4. 병합
df_all = df_border.merge(df_un, on='국가')\
                  .merge(df_org, on='국가')\
                  .merge(df_trade, on='국가')\
                  .merge(df_terror, on='국가')

# 5. 결측치 처리 (있다면)
df_all = df_all.dropna()

# 6. 분석 변수 및 표준화
features = ['테러발생건수', '국제무역규모', '지역기구활동점수', 'UN평화유지군', '국경공유국수']
X = df_all[features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 7. PCA (PC1 추출)
pca = PCA(n_components=1)
pc1 = pca.fit_transform(X_scaled)
loadings = pca.components_[0]

# 8. 가중치 정규화 (합=1, 절대값합=1)
loadings_sum_norm = loadings / loadings.sum()
loadings_abs_norm = loadings / np.abs(loadings).sum()
directions = np.where(loadings > 0, '긍정적', '부정적')

# 9. PC1 점수 0~1 정규화
minmax = MinMaxScaler()
pc1_norm = minmax.fit_transform(pc1)

# 10. 결과 데이터프레임
df_result = pd.DataFrame({
    '국가': df_all['국가'],
    '지정학적가치지수': pc1_norm.flatten()
})

df_weights = pd.DataFrame({
    '지표': features,
    'PCA_가중치(loading)': loadings,
    '정규화가중치(합=1)': loadings_sum_norm,
    '정규화가중치(절대값합=1)': loadings_abs_norm,
    '방향성': directions
})

# 11. 저장 및 다운로드
df_result.to_csv('geopolitical_index.csv', index=False, encoding='utf-8-sig')
df_weights.to_csv('geopolitical_weights.csv', index=False, encoding='utf-8-sig')

files.download('geopolitical_index.csv')
files.download('geopolitical_weights.csv')


Saving 아프리카_테러발생건수_20112022.csv to 아프리카_테러발생건수_20112022 (2).csv
Saving 아프리카_25개국_UN평화유지군_파견인원_20112022.csv to 아프리카_25개국_UN평화유지군_파견인원_20112022 (2).csv
Saving 아프리카_25개국_국제무역규모_20112022.csv to 아프리카_25개국_국제무역규모_20112022 (2).csv
Saving 아프리카_국경공유국수_20112022.csv to 아프리카_국경공유국수_20112022 (2).csv
Saving 아프리카_지역기구활동점수_20112022.csv to 아프리카_지역기구활동점수_20112022 (2).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>